### Подключение библиотек и скриптов

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, train_test_split

### 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Выбрал Statlog (German Credit Data) Data Set

#### Загрузка данных

In [3]:
data = pd.read_csv("SouthGermanCredit.asc", sep=' ')

### 3. сделать feature engineering

К полям:
-  применим OHE-кодирование
- laufzeit, hoehe, alter - standardScaler
- gastarb, telef, pers оставим пока как есть, laufkont, moral, verw, sparkont, beszeit, rate, famges, buerge, wohnzeit, verm, weitkred, wohn, bishkred, beruf тоже оставим

In [4]:
x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler

base_cols = ['laufkont', 'moral', 'verw', 'sparkont', 'beszeit', 'rate', 'famges', 'buerge', 'wohnzeit',
              'verm', 'weitkred', 'wohn', 'bishkred', 'beruf']
continuos_cols = ['laufzeit', 'hoehe', 'alter']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
# for cat_col in cat_cols:
#     cat_transformer = Pipeline([
#                 ('selector', ColumnSelector(key=cat_col)),
#                 ('ohe', OHEEncoder(key=cat_col))
#             ])
#     cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [6]:
from sklearn.pipeline import FeatureUnion

# feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feats = FeatureUnion(continuos_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 1.23159172e+00,  1.79151936e+00, -8.22306639e-01, ...,
         2.00000000e+00,  1.00000000e+00,  3.00000000e+00],
       [ 2.56154007e-01, -4.31285795e-01,  1.64371630e-01, ...,
         2.00000000e+00,  1.00000000e+00,  3.00000000e+00],
       [ 2.20702943e+00,  1.14226114e+00,  2.13772817e+00, ...,
         1.00000000e+00,  1.00000000e+00,  3.00000000e+00],
       ...,
       [-1.36957551e+00, -6.34335662e-01,  6.12861752e-01, ...,
         2.00000000e+00,  3.00000000e+00,  2.00000000e+00],
       [-1.20700256e+00,  1.26581000e+00,  8.81955825e-01, ...,
         2.00000000e+00,  2.00000000e+00,  4.00000000e+00],
       [ 2.56154007e-01,  3.10483659e-03,  7.46736053e-02, ...,
         2.00000000e+00,  1.00000000e+00,  4.00000000e+00]])

### 4. обучить любой классификатор (какой вам нравится)

In [7]:
from catboost import CatBoostClassifier

In [8]:
# Функция обучения модели и расчета метрик
def get_metrics(classifier):
    
    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]

In [9]:
frozen_params = {
    'eval_metric': 'F1',
    'auto_class_weights': 'Balanced',
    'silent': True,
    'one_hot_max_size': 20,
    'early_stopping_rounds': 50,
    'boosting_type': 'Ordered',
    'allow_writing_files': False
}

classifier_4 = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(
    **frozen_params,
    depth=4,
    iterations=400,
    learning_rate=0.1,
    l2_leaf_reg=2.5,
    bagging_temperature=1.5
)),
])

In [10]:
get_metrics(classifier_4)

#### Проверяем качество

CatBoostClassifier показывает лучший рез-т по precision, согласно условиям задачи "It is worse to class a customer as good when they are bad (5), than it is to class a customer as bad when they are good (1)." рез-т по precision более важен чем рез-т по recall

In [11]:
classifier_4.steps

[('features',
  FeatureUnion(transformer_list=[('laufzeit',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='laufzeit')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('hoehe',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='hoehe')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('alter',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='alter')),
                                                  ('standard',
                                                   StandardScaler())])),
           

In [12]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [13]:
import dill
with open("german_credit_data_pipeline.dill", "wb") as f:
    dill.dump(classifier_4, f)

In [14]:
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)